## [Missing Semester] 코로나19 확진자수
- 공공데이터활용지원센터_보건복지부 코로나19 감염 현황
- https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043376


- requests : Python 환경에서 HTTP Requests
- xmltodict : XML 데이터를 Dict로 변환 후 다시 DataFrame으로 변환
- DataFrame.diff() : 일별 누적 확진자 수를 일일 확진자 수로 변환


In [ ]:
!pip install xmltodict

In [ ]:
# -*- coding: utf-8 -*-
import requests
import xmltodict
import pandas as pd 

**requests : url로 데이터 받아오기(이 부분이 과제입니다!)**

In [ ]:
url_base = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson"
serviceKey = "" # ""안에 일반 인증키(Encoding) 입력
pages = "" # 페이지당열갯수 입력
start_date = "" # 시작 날짜 입력
end_date = "" # 끝 날짜 입력
url = f"{url_base}?serviceKey={serviceKey}&pageNo=1&numOfRows={pages}&startCreateDt={start_date}&endCreateDt={end_date}"

print(url)

In [ ]:
response = requests.get(url).content # get request를 날려 돌아온 response를 response 변수에 저장

# API를 가져오는건 여기까지 사전지식으로 가능한 실습
# 아래부터는 받아온 데이터를 활용하는 단계

**xmltodict : XML to DataFrame**

In [ ]:
xmltodictObject = xmltodict.parse(response) # 문자열을 XML을 Dict 형식으로 변환합니다.
dict_data = xmltodictObject['response']['body']['items']['item'] # 필요한 데이터만 선택해서 불러오기
print(dict_data) # OrderedDict는 데이터가 삽입된 순서로 존재하는 dictionary

In [ ]:
data_frame = pd.DataFrame(dict_data) # 선택한 데이터로 dataframe 객체 생성 (2차원 구조)

In [ ]:
data_frame.head() # 함수를 이용하여 상위 5행을 출력합니다.

In [ ]:
data_frame = data_frame.astype({'decideCnt' : 'int', 'examCnt' : 'int', 'deathCnt' : 'int'}) # Dataframe에서 사용할 데이터의 타입을 변경

In [ ]:
print(data_frame.dtypes) # 타입 변경 후 확인

In [ ]:
data_frame = data_frame.drop_duplicates(['stateDt']) # 중복제거: 하루에 2회 이상 중복 확진자 발표가 있을 경우 마지막 발표한 것으로 대체 

In [ ]:
data_frame['date'] = data_frame['stateDt']
data_frame['date'] = pd.to_datetime(data_frame['date']) # api로 받아와 DataFrame에 저장하면 날짜데이터의 형식은 int64로 지정되어 저장됩니다. 이를 datetime64형식으로 변환
data_frame.set_index('date') # set_index() 메서드를 사용해 DataFrame에 설정된 컬럼명(열)을 인덱스로 사용

**DataFrame.diff()**

In [ ]:
data_frame_2 = data_frame[['date','decideCnt','examCnt','deathCnt']] # 날짜, 확진자수, 검사진행 수, 사망자 수

In [ ]:
data_frame_2 = data_frame_2.sort_values(by='date') #diff()를 사용하기 위해서 날짜로 오름차순 정렬

In [ ]:
data_frame_2['daily_confirmedCases'] = data_frame_2['decideCnt'].diff() # 당일 누적 데이터 - 전일 누적 데이터 = 일일 확진자 데이터

In [ ]:
data_frame_2

In [ ]:
data_frame_2.plot('date', ['daily_confirmedCases'])

### 요약
- requests : Python 환경에서 HTTP Requests
- xmltodict : XML 데이터를 Dict로 변환 후 다시 DataFrame으로 변환
- DataFrame.diff() : 일별 누적 확진자 수를 일일 확진자 수로 변환